In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import stock_automation_utils as SAU
import pickle
from datetime import datetime, timedelta
import numpy as np

In [ ]:
import time, os
import yfinance as yf
import pandas as pd
from tqdm import tqdm

In [ ]:
stock_symbols = ['^IXIC']
throttle_secs = 0
data_folder='data'

In [ ]:
# #Update data for all stocks.
# SAU.load_and_update_stock_data()

In [ ]:
stock_symbols_pickle = 'stock_symbols.pkl'  # Name of the pickle file

In [ ]:
# Load symbols into the stock_symbols.pkl
symbols = []  # Example list of stock symbols
if len(symbols) > 0:
    SAU.add_stock_symbols_to_pickle(symbols, stock_symbols_pickle)

In [ ]:
# Load the data from 'stock_symbols.pkl' into a variable called 'symbols'
with open(stock_symbols_pickle, 'rb') as file:
    symbols = pickle.load(file)

## BACKTEST

In [ ]:
start_date = '2018-01-01'
end_date = '2020-01-31'
start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
rebalance_frequency = 15  # In days
# create a numpy array of numbers 15 days apart from 0 to 365
days_ago_list = list(range(0, 365, rebalance_frequency))
long_count = 10
short_count = 0
portfolio_starting_value = 110000
risk_pct = 0.10

In [ ]:
import stock_automation_utils as SAU
import backtest_utils as BU
backtest_runs, backtest_profits = BU.run_backtest(symbols, start_date_dt, end_date_dt, rebalance_frequency, days_ago_list, long_count, short_count, portfolio_starting_value, risk_pct)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sum(backtest_profits)

In [ ]:
# Bring in the Nasdaq data and calculate the returns
nasdaq_data = SAU.get_stock_data('^IXIC', start_date, end_date)


In [ ]:
# Only keep the Adjussted Close column
nasdaq_data = nasdaq_data[['Date', 'Adj Close']]
# Convert 'Date' to datetime
nasdaq_data['Date'] = pd.to_datetime(nasdaq_data['Date'])
rebalance_periods = BU.calculate_rebalance_periods_with_dates(start_date_dt, end_date_dt, rebalance_frequency)
rebalance_periods = [(datetime.combine(rebalance_periods[0], datetime.min.time()), datetime.combine(rebalance_periods[1], datetime.min.time())) for rebalance_periods in rebalance_periods]
dates = np.array(rebalance_periods)[:, 1]
# For each date in dates, find a date in nasdaq_data that is the same or the closest
nasdaq_returns = []
for date in dates:
    idx = nasdaq_data['Date'].sub(date).abs().idxmin()
    nasdaq_returns.append(nasdaq_data.loc[idx, 'Adj Close'])



In [ ]:
len(backtest_profits), len(rebalance_periods), len(nasdaq_returns)

In [ ]:
nasdaq_returns = np.array(nasdaq_returns)

In [ ]:
benchmark_data = ((1-(nasdaq_returns[1:]/nasdaq_returns[:-1]))*portfolio_starting_value) * -1
benchmark_data = np.insert(benchmark_data, 0, 0)


In [ ]:
# add a zero to the beginning of the benchmark_data to make it the same length as the backtest_profits
len(backtest_profits), len(rebalance_periods), len(nasdaq_returns), len(benchmark_data)

In [ ]:
# Plot the backtest profits and the benchmark_data
plt.figure(figsize=(16, 9))
plt.plot(rebalance_periods, np.cumsum(backtest_profits), label='Backtest Profits')
plt.plot(rebalance_periods, np.cumsum(benchmark_data), label='Nasdaq Returns')
# Show labels
plt.legend()
# Show the plot
plt.show()

In [ ]:
plt.plot(backtest_profits)

In [ ]:
profit_count = 0
loss_count = 0
for backtest_run in backtest_runs:
    profit = backtest_run[-1]
    for output in profit:
        output = int(np.float32(output))
        if output > 0:
            profit_count += 1
        else:
            loss_count += 1
print({'profit_count': profit_count, 'loss_count': loss_count, 'profit_pct': profit_count/(profit_count+loss_count)})

In [ ]:
# calculate the sharp ratio
# add portfolio starting value as the first element of the backtest_profits
backtest_profits_temp = np.insert(backtest_profits, 0, portfolio_starting_value)
# https://www.youtube.com/watch?v=vTzjk6kLw2I

In [ ]:
backtest_profits_temp

In [ ]:
np.cumsum(backtest_profits_temp)

# Analysis of the backtest

In [ ]:
'''
# Add nasdaq graph to the plot - I want to find out on a week to week basis, how much money I would have made if I had just bought the Nasdaq
# On a week to week basis, % return of the strategy compared to the Nasdaq (eg. strategy made 10%, Nasdaq made 5%, answer is 5%. If strategy made 5% and Nasdaq made 10%, answer is -5%)
# Compound Annual Growth Rate (CAGR) - the mean annual growth rate of an investment over a specified time period longer than one year
# Sharpe Ratio - measures the performance of an investment compared to a risk-free asset, after adjusting for its risk
# Maximum Drawdown - the maximum loss from a peak to a trough of a portfolio
# Beta (measure of volatility) - how much the stock moves in relation to the market
# Alpha (measure of performance on a risk-adjusted basis)
# Win/Loss Ratio, Avg Win, Avg Loss
# Profit Factor - the ratio of total profit to total losses

'''

In [ ]:
# calculate the rolling max of a numpy array
def rolling_max(arr, window):
    return np.maximum.accumulate(arr)


In [ ]:
# calculate the median of all +ve and -ve numbers in a numpy array
def median(arr):
    arr = np.array(arr)
    pos = arr[arr > 0]
    neg = arr[arr < 0]
    return np.median(pos), np.median(neg)

In [ ]:
median(backtest_profits)

# IBKR Connection

In [ ]:
from ib_insync import IB, Stock
import nest_asyncio
nest_asyncio.apply()

In [ ]:
def connect_to_IBKR():
    # Create an instance of the IB class
    ib = IB()

    # Connect to the IBKR TWS (Trader Workstation) or Gateway
    ib.connect('127.0.0.1', 7496, clientId=0)

    # Check if the connection is successful
    if ib.isConnected():
        print('Connected to IBKR')
    else:
        print('Failed to connect to IBKR')
    return ib

In [ ]:
def get_current_portfolio(ib):
    # Request the current portfolio from IBKR
    portfolio = ib.portfolio()
    
    # Return the portfolio
    return portfolio


In [ ]:
def get_current_price(ib):
    # Request the current market data for AAPL
    contract = ib.qualifyContracts(Stock('AAPL', 'SMART', 'USD'))
    ticker = ib.reqTickers(contract)[0]
    
    # Get the current price
    current_price = ticker.marketPrice()
    
    # Return the current price
    return current_price


In [ ]:
get_current_price(ib)